# Topic Assignment for the DS Project

Thhis file conducts the topic assignment for the lyrics analytics project based on SBBERT models. 
The file for the similarity Scores outgh to be executed before this file!


In [ ]:
!pip install transformers
!pip install sentence-transformers

     |████████████████████████████████| 3.4 MB 26.9 MB/s 
     |████████████████████████████████| 67 kB 8.5 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 895 kB 77.6 MB/s 
     |████████████████████████████████| 3.3 MB 52.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 1.2 MB 64.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=da77cd47c58bded17a931a0fee2dccabc2b7de8310792f563886114d0e5fb326
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
import torch 
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np 


In [ ]:
## mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# using a pre-trained Sentence Transformer BERT Model 
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')  ##  'bert-base-nli-mean-tokens')  # using the mean tokens


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

## defining characaters strings best describing the different topics

In [ ]:
topic_love = ["Love, romance and sex"]

topic_friends_party = ["Friends, parties and dancing"]

topic_money_fame = ["Money, fame and affluence"]

topic_sadness = ["Anger, critique and sadness"]

topic_motivation = ["Motivation and ambition"]


In [ ]:
## list of names of topics 

list_topics = ["Love and Romance", "Feelgood, friends and Party", "Affluence and Fame", "Sadness and Critique", "Motivation and Ambitions"]

In [ ]:
## reading in data

song_data =  pd.read_csv("/content/gdrive/My Drive/DS Projekt/data_for_BERT.csv",  encoding='latin-1')

song_data.head()


,genre,combination,lyrics,first_appearance
0,country,Marty Robbins El Paso,Out in the West Texas town of El Paso I fell i...,1960-01-02
1,pop,Frankie Avalon Why,I ll never let you go Why Because I love you ...,1960-01-02
2,pop,Johnny Preston Running Bear,On the bank of the river Stood Running Bear y...,1960-01-02
3,pop,Freddie Cannon Way Down Yonder In New Orleans,Well way down yonder in New Orleans In the la...,1960-01-02
4,pop,Guy Mitchell Heartaches By The Number,Heartaches by the number troubles by the scor...,1960-01-02


In [ ]:

all_lyrics = list(song_data.lyrics)

In [ ]:
# getting the embeddings for all songs

all_lyrics = list(song_data.lyrics)
print("The amount of songs considered for the Sentence BERT Model is", str(len(all_lyrics)), "!")

The amount of songs considered for the Sentence BERT Model is 24585 !


In [ ]:
## to GPU


device = torch.device("cuda")
model = model.to(device)

In [ ]:
all_embeddings = model.encode(all_lyrics)

In [ ]:
np.array(all_embeddings).shape

(24585, 768)

In [ ]:
## embeddings of description vectors

all_descriptions = list([topic_love, topic_friends_party, topic_money_fame, topic_sadness, topic_motivation])


In [ ]:
all_descriptions =[y for x in all_descriptions for y in x]

In [ ]:
des_embeddings = model.encode(all_descriptions)    


In [ ]:
from sentence_transformers.util import cos_sim

#sim = np.zeros([embeddings.shape[0], embeddings.shape[0] ])


In [ ]:
topics = []
topics_2 = []


In [ ]:
all_embeddings2 = np.array(all_embeddings)
des_embeddings2 = np.array(des_embeddings)

In [ ]:
all_embeddings2.shape

(24585, 768)

In [ ]:
des_embeddings2.shape

(7, 768)

In [ ]:

for i in range(all_embeddings2.shape[0]):
  

  #embeddings.shape[0]):
  list_sims = np.ones(len(all_descriptions))

  print(i)
  for j in range(len(all_descriptions)):
    list_sims[j] = cos_sim(  all_embeddings2[i], des_embeddings2[j])

  # maximum 
  max_index = np.argmax(list_sims)

  # second 
  list_sims[max_index] = - 100
  max_index_2 = np.argmax(list_sims)

  # appending 
  topics.append(list_topics[max_index])
  topics_2.append(list_topics[max_index_2])
  

  if i == all_embeddings2.shape[0]:
    df_topics = pd.DataFrame({"topics":topics, "topics_2":topics_2})
    df_topics.to_csv('/content/gdrive/My Drive/DS Projekt/df_topics.csv')

  if (i > 0) & ((i % 400) == 0):
    df_topics = pd.DataFrame({"topics":topics, "topics_2":topics_2})
    df_topics.to_csv('/content/gdrive/My Drive/DS Projekt/df_topics.csv')

#sim.shape

## Checking if the topic classification made sense

In [ ]:
# check if it has worked # part  1

for i in range(10):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


Out in the West Texas town of El Paso I fell in love with a Mexican girl Nighttime would find me in Rosa s Cantina Music would play and Feleena would whirl Blacker than night were the eyes of Feleena Wicked and evil while casting her spell My love was deep for this Mexican maiden I was in love  but in vain  I could tell One night  a wild young cowboy came in Wild as the West Texas wind Dashing and daring  a drink he was sharing With wicked Feleena  the girl that I love So in anger  I challenged his right for the love of this maiden Down went his hand for the gun that he wore My challenge was answered in less than a heartbeat The handsome young stranger lay dead on the floor Just for a moment I stood there in silence Shocked by the foul evil deed I had done Many thoughts raced through my mind as I stood there I had but one chance and that was to run Out through the back door of Rosa s  I ran Out where the horses were tied I caught a good one  it looked like it could run Up on its back a

0-10 wors really nicely!

In [ ]:
# check if it has worked # part  2

for i in range(4000,4010):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


Thou shalt never love another And stand by me all the while Take happiness with the heartache And go through life wearing a smile Oh  how happy we would be if we Keep the ten commandments Of love  of love Thou should always have faith in me In everything I say and do Love me with all your heart and soul Until our life on earth is through Oh  how happy we would be if we Keep the ten commandments of love Love  oh  sweet love It s oh  so grand You will find since The beginning of time It has ruled in all the land Come to me when I am lonely Kiss me when you hold me tight Treat me sweet and gentle When we say goodnight Oh  how happy we would be if we Keep the ten commandments Of love  of love Oh  how happy we would be if we Keep the ten commandments of love Love  ooh  ooh  ooh Thanks to razvan
Love and Romance
Sadness and Critique
Soulville  soulville Come on and Show me the way To get to soulville  baby Show me the way to go home  The way to go home  Show me the way To get to soulville  b

4000-4010 works well as well!

In [ ]:
# check if it has worked # part  3

for i in range(10000,10010):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


I know a guy who s tough but sweet He s so fine  he can t be beat He s got everything that I desire Sets the summer sun on fire I want candy I want candy Go to see him when the sun goes down Ain t no finer boy in town You re my guy  you re what the doctor ordered So sweet  you make my mouth water I want candy I want candy Yeah  Candy on the beach  there s nothing better But I like candy when it s wrapped in a sweater Some day soon I ll make you mine Then I ll have candy all the time I want candy I want candy I want candy I want candy Hey  Hey  Hey  Hey  Hey 
Love and Romance
Feelgood, friends and Party
If you ever Plan to motor west Travel my way  take the highway that s the best Get your kicks on Route 66 It winds from Chicago to LA More than two thousand miles all the way Baby  get your kicks on Route 66 Now you go through St  Louie Joplin  Missouri And Oklahoma City looks mighty pretty You ll see Amarillo Gallup  New Mexico Flagstaff Arizona Don t forget Winona Kingman  Barstow  San

Works nicely!

In [ ]:
# check if it has worked # part  4

for i in range(15000,15010):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


You re the devil in me I brought in from the cold You said your body was young But your mind was very old You re coming on strong and I like the way The visions we had have faded away You re part of a life I ve never had I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad You re coming on strong You re coming on strong You re showing your color Like a setting sun You re the devil in me I brought in from the cold You said your body was young But your mind was very old You re coming on strong and I like the way The visions we had have faded away You re part of a life I ve never had I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad I ll tell you that it s just too bad
Sadness and Critique
Feelgood, friends and Party
Is it any wonder I hold you so tight It s beca

Works really well, exampe hof cathing Motivation topic!

In [ ]:
# check if it has worked # part  5

for i in range(20000,20010):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


Fix a Heart    It s probably what s best for you   I only want the best for you   And if I m not the best  then you re stuck   I tried to sever ties and I ended up with wounds to bind   Like you re pouring salt in my cuts   And I just ran out of bandaids   I don t even know where to start    Cause you can bandage the damage   You never really can fix a heart   Even though I know what s wrong   How could I be so sure if you never say what you feel  feel    I must ve held you hand so tight   You didn t have the will to fight   I guess you needed more time to heal   Baby  I just ran out of bandaids   I don t even know where to start    Cause you can bandage the damage   You never really can fix a heart   Ooh  oh  oh   Oh  oh  yeah   Ooh  oh  oh   Oh  oh  oh  yeah   You must be a miracle worker   Swearing up and down  you can t fix what s been broken  yeah   Please don t get my hopes up   No  no   Baby  tell me  how could you be so cruel    It s like you re pouring salt on my cuts   Baby  

Nice! :D

In [ ]:
# check if it has worked # part  1

for i in range(len(topics)-10,len(topics)):
  print(song_data.lyrics[i])
  print(topics[i])
  print(topics_2[i])
  


I got that big  big energy Got that real big energy Hey  daddy When you gon  stop playin   Bad bitch  I could be your fantasy  Yeah  I can tell you got big dick energy  Uh huh  It ain t too many niggas that can handle me  Yeah  But I might let you try it off the Hennessy  Uh  Make  em sing to this pussy like a melody  Make  em sang  And if your bitch ain t right  I got the remedy  Ooh  It ain t too many niggas that can handle me  Ayy  Bad bitch  I could be your fantasy  Ayy  ayy  Tell me how you want it  Huh   Three  two  one and I m on it  On it  Feel good  don t it   Don t it   Hood bitch  fuck you in a bonnet  Yeah  I ma bust it on a pole like onyx  Ooh  I m just bein  honest  Uh huh  Pussy juicy  Minute Maid  Oh  But can t do it one minut U 0435   man  Hell no  Not a side or a main I m the only bitch h U 0435  entertain  Yeah  Spendin  his mine in the bank  In the bank  I like what I see  Yeah  A boss like you need a boss like me  Uh huh  Daddy from the street  so he move lowkey Tr

## Saving topics to Drive and merging with ssimilarity DF!

In [ ]:
df_topics = pd.DataFrame({"topics":topics, "topics_2":topics_2})


In [ ]:
df_topics.to_csv('/content/gdrive/My Drive/DS Projekt/df_topics.csv')

In [ ]:
song_data["topics"] = topics
song_data["topics_2"] = topics_2


In [ ]:
## reading in similarity data 

similarity_df = pd.read_csv('/content/gdrive/My Drive/DS Projekt/similarity.csv')



In [ ]:

similarity_df.head()

,Unnamed: 0,X1,s1,s2,s3,genre,combination,first_appearance,p1,p2
0,1,0,Theory Of A Deadman Bad Girlfriend,Toby Keith Bullets In The Gun,Brantley Gilbert My Baby's Guns N' Roses,country,Marty Robbins El Paso,1960-01-02,-4.539968,1.974627
1,2,1,The Beatles Love Me Do,Etta James My Dearest Darling,Al Martino I Love You Because,pop,Frankie Avalon Why,1960-01-02,8.582279,4.375166
2,3,2,Sonny James Running Bear,Jo Ann Campbell I'm The Girl From Wolverton M...,Justin Moore Backwoods,pop,Johnny Preston Running Bear,1960-01-02,-2.297034,-0.219208
3,4,3,Lo-Key? I Got A Thang 4 Ya!,Corinne Bailey Rae Put Your Records On,Ritt Momney Put Your Records On,pop,Freddie Cannon Way Down Yonder In New Orleans,1960-01-02,0.000069,5.427196
4,5,4,Carly Pearce & Lee Brice I Hope You're Happy Now,The Vejtables I Still Love You,Johnny Tillotson Heartaches By The Number,pop,Guy Mitchell Heartaches By The Number,1960-01-02,3.771206,-3.120906


In [ ]:
## merging 

similarity_topics_df = similarity_df.merge(song_data[["combination","topics", "topics_2"]], on = "combination", how = "left")

In [ ]:
similarity_topics_df.head()

,Unnamed: 0,X1,s1,s2,s3,genre,combination,first_appearance,p1,p2,topics,topics_2
0,1,0,Theory Of A Deadman Bad Girlfriend,Toby Keith Bullets In The Gun,Brantley Gilbert My Baby's Guns N' Roses,country,Marty Robbins El Paso,1960-01-02,-4.539968,1.974627,Sadness and Critique,Love and Romance
1,2,1,The Beatles Love Me Do,Etta James My Dearest Darling,Al Martino I Love You Because,pop,Frankie Avalon Why,1960-01-02,8.582279,4.375166,Love and Romance,Sadness and Critique
2,3,2,Sonny James Running Bear,Jo Ann Campbell I'm The Girl From Wolverton M...,Justin Moore Backwoods,pop,Johnny Preston Running Bear,1960-01-02,-2.297034,-0.219208,Sadness and Critique,Love and Romance
3,4,3,Lo-Key? I Got A Thang 4 Ya!,Corinne Bailey Rae Put Your Records On,Ritt Momney Put Your Records On,pop,Freddie Cannon Way Down Yonder In New Orleans,1960-01-02,0.000069,5.427196,Love and Romance,Affluence and Fame
4,5,4,Carly Pearce & Lee Brice I Hope You're Happy Now,The Vejtables I Still Love You,Johnny Tillotson Heartaches By The Number,pop,Guy Mitchell Heartaches By The Number,1960-01-02,3.771206,-3.120906,Sadness and Critique,Love and Romance


In [ ]:
similarity_topics_df.to_csv('/content/gdrive/My Drive/DS Projekt/similarity_topics_df.csv')